## OpenAI에서 지원하는 Assistants API 를 활용하여 PPT 파일을 생성해보자
- Assistants: 대화 및 파일 생성, 코드 명령 수행, 데이터 검색 추출 등 특정 task 에 특화되어 학습된 자연어 기반 생성형 인공지능 API
- OpenAI의 일반적인 ChatAPI처럼 대화형 서비스도 제공하지만 특정 작업에 특화하여 지원하거나 자동화에 중점을 둠으로써 챗봇, 가상비서, 자동응답시스템 개발 등 필요한 작업에 포커스를 맞춰 사용가능

## 1. OpenAI API key 설정

In [17]:
from openai import OpenAI
from getpass import getpass
import time

In [4]:
MY_API_KEY=getpass('OpenAI API KEY:')

OpenAI API KEY: ········


In [16]:
client=OpenAI(api_key=MY_API_KEY)

## 2. 어시스턴트, 스레드, 메세지 객체 설정
- 스레드(thread): 프로세스 내에서 실행되는 가장 작은 실행단위로 현재 실습에서는 어시스턴트와 사용자간 채팅 세션(공간)을 하나의 스레드로 지정하여 진행
- 메세지(message): 스레드 내의 통신 단위

<진행 순서>
1) 어시스턴트 세부사항 정의
2) 어시스턴트 객체 생성
3) 스레드 및 메세지 객체 생성
4) 스레드 실행(대화 시작 및 요구사항 요청)

### 1) 어시스턴트 세부사항 정의

In [20]:
#어시스턴트의 역할을 정의(role에서 system과 유사한 기능)
assistant_instruction="""등산에 관련된 powerpoint 파일을 만들어야해. 
너는 등산전문가이자 파워포인트 전문가야. 전체적인 PPT의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해줘. 
페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point 정도로 설정해줘. 
슬라이드는 3개 만들어줘"""

#원하는 요청을 작성
prompt_user="""입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘. 
초급, 중급, 고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 포함하는 프레젠테이션을 만들어줘. 페이지 구성이 깔끔하고 내용은 구체적으로 작성해주면 좋겠어."""

### 2) 어시스턴트 객체 생성

In [22]:
assistant = client.beta.assistants.create(name= 'My Assistant', #이름은 원하는 대로 지정
                                         instructions =assistant_instruction,
                                         #현재 tools에 적용 가능한 타입은 code_interpreter, fuction, file_search 등이 있음
                                         tools=[{'type':'code_interpreter'}],
                                         model='gpt-4o'
                                        )

#code_interpreter: 모델이 코드를 실행하여 계산, 데이터분석, 파일조작 등의 작업을 수행
#fuction: 특정 작업을 구현할 수 있는 사용자 정의함수를 호출하여 작업 수행
#file_search: 사전 구성된 환경에서 파일을 검색하고 가져와 특정 작업 수행

In [23]:
assistant


Assistant(id='asst_zEDpNcxYVMUvEdhhfoUe8fDu', created_at=1738635043, description=None, instructions='등산에 관련된 powerpoint 파일을 만들어야해. \n너는 등산전문가이자 파워포인트 전문가야. 전체적인 PPT의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해줘. \n페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point 정도로 설정해줘. \n슬라이드는 3개 만들어줘', metadata={}, model='gpt-4o', name='My Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)

### 3) 스레드 및 메세지 객체 생성

In [25]:
#스레드 객체 생성
 #대화를 시작하면 스레드가 생성되며 대화중 교환되는 모든 메세지는 동일한 스레드에서 진행됨
thread=client.beta.threads.create()

#매세지 객체 생성
 #메세지는 사용자의 질의, 명령과 같은 텍스트 뿐만 아니라 파일 등 모든 컨텐츠가 포함됨
 #메세지는 스레드 내의 통신 단위이므로 위에서 생성한 스레드의 id값을 입력
message=client.beta.threads.messages.create(thread_id=thread.id,
                                            role='user',
                                            content=prompt_user
                                           )

In [27]:
thread

Thread(id='thread_pUCuXuVAD811kdWmU3bxewgz', created_at=1738635372, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [28]:
message

Message(id='msg_ATf60joOdArxKMpVcYbOQTxz', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘. \n초급, 중급, 고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 포함하는 프레젠테이션을 만들어줘. 페이지 구성이 깔끔하고 내용은 구체적으로 작성해주면 좋겠어.'), type='text')], created_at=1738635372, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pUCuXuVAD811kdWmU3bxewgz')

### 4) 스레드 실행(대화 시작 및 요구사항 요청)
- 위에 설정된 정보로 역할 설정 및 원하는 작업을 요청하는 단계

In [38]:
#스레드 실행(실행하면 LLM으로 사용자의 요청이 넘어감)
#스레드는 사용자가 요청을 입력했을때 시작되고 모델의 응답이 끝나면 종료됨

run = client.beta.threads.runs.create(thread_id=thread.id,
                                      assistant_id=assistant.id
                                     )
                                      

- 스레드를 실행시킨다고 설정해 둔 모든 요청과 응답이 즉각적으로 이루어지지는 않음
- 생성 모델에 사용자의 요청을 인지하고 결과물을 생성하기 위해서는 어느정도의 시간이 필요함(복잡하고 어려운 요청일 수록 더 시간이 오래걸릴 수 있음)

### ###assistants 응답진행상황 확인 코드###
- 작업을 완료하는 데 충분한 시간이 흘렀다면 실행 완료가 뜨고, 실행중이라면 실행되고 있는 과정이 체크되도록 코드 작성

In [39]:
while True:
    #retrieve:특정 스레드의 실행상태나 실행후 결과를 검색하는 데 사용하는 함수
    run_retrieve=client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id
                                                 )
    #실행완료인 경우
     #status는 in_progress, completed, failed로 출력될 수 있음
    if run_retrieve.status =='completed':
        print('작업 완료')
        break
    #작업이 실행중이거나 실패한 경우
    else:
        print(run_retrieve.status)
        #일정한 시간 텀을 두고 확인(서버에 부하가 가는 것을 방지하기 위함)
        time.sleep(1)

in_progress
in_progress
작업 완료


In [41]:
#messages.list: 특정스레드의 메세지에 대한 정보와 그 목록을 보여주는 함수
messages=client.beta.threads.messages.list(thread_id=thread.id)
messages

#맨 아래에서부터 사용자의 요청메세지와 그에대한 모델의 응답메세지가 순차적으로 출력되고 있음

SyncCursorPage[Message](data=[Message(id='msg_pBWjZS02nY4PA1Mzk5OEbDtG', assistant_id='asst_zEDpNcxYVMUvEdhhfoUe8fDu', attachments=[Attachment(file_id='file-4nN6Vp6XcYSyc5kqF2Nx9E', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=144, file_path=FilePath(file_id='file-4nN6Vp6XcYSyc5kqF2Nx9E'), start_index=102, text='sandbox:/mnt/data/hiking_presentation.pptx', type='file_path')], value='프레젠테이션을 성공적으로 만들었습니다. 등산 입문자 및 초보 등산가를 위한 강의 자료인 파워포인트 파일을 다운로드 하실 수 있습니다: [hiking_presentation.pptx](sandbox:/mnt/data/hiking_presentation.pptx)'), type='text')], created_at=1738636828, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ehu7pblMAcmEGXxV5h64ygSp', status=None, thread_id='thread_pUCuXuVAD811kdWmU3bxewgz'), Message(id='msg_vuRSUANKXq9TII94bgYqCVu8', assistant_id='asst_zEDpNcxYVMUvEdhhfoUe8fDu', attachments=[], completed_at=No

In [45]:
messages.data[0].content[0].text

Text(annotations=[FilePathAnnotation(end_index=144, file_path=FilePath(file_id='file-4nN6Vp6XcYSyc5kqF2Nx9E'), start_index=102, text='sandbox:/mnt/data/hiking_presentation.pptx', type='file_path')], value='프레젠테이션을 성공적으로 만들었습니다. 등산 입문자 및 초보 등산가를 위한 강의 자료인 파워포인트 파일을 다운로드 하실 수 있습니다: [hiking_presentation.pptx](sandbox:/mnt/data/hiking_presentation.pptx)')

In [46]:
#annotations: 메세지의 유형텍스트, 이미지, 비디오, 각종 파일)에 대한 정보가 담겨져 있음
messages.data[0].content[0].text.annotations

[FilePathAnnotation(end_index=144, file_path=FilePath(file_id='file-4nN6Vp6XcYSyc5kqF2Nx9E'), start_index=102, text='sandbox:/mnt/data/hiking_presentation.pptx', type='file_path')]

#### annotations가 비어있는 경우의 원인
- 생성모델이 결과물을 전부 생성하기 전에 코드를 실행한 경우(충분한 시간이 지나고 실행하면 됨)
- 필요한 데이터가 메세지에 포함되지 않았을 경우(파일 미첨부, 애매한 지시 등)
- API 호출에 문제가 있거나 잘못된 thread_id를 사용하여 올바른 데이터를 가져오지 못한 경우

In [52]:
#생성한 파일의 정보 추출

#메세지를 통해 받은 파일 정보 중 생성한 파일(PPT)의 file_id 값을 변수에 저장
file_id_path=messages.data[0].content[0].text.annotations[0].file_path.file_id

#retrieve_content: file_id값을 통해 생성된 파일의 내용물을 검색해 가져오는 함수
file_contents=client.files.with_raw_response.retrieve_content(file_id_path)

### 결과물을 실제 파일로 내보내기


In [53]:
#with open(경로 및 파일명, 읽기 및 쓰기 설정) 파일 불러오기 및 내보내기 기능 코드
with open('result/PPT_001.pptx','wb') as f:
    #메세지에서 가져온 파일의 내용을 바탕으로 실제 생성할 파일에 컨텐츠 쓰기
    f.write(file_contents.content)

In [ ]:
### 생성파일 정보 및 어시스턴트 삭제(메모리 끊기)

In [54]:
client.files.delete(file_id_path)
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_zEDpNcxYVMUvEdhhfoUe8fDu', deleted=True, object='assistant.deleted')

### 정리
- 간단한 요청이었기에 결과물의 퀄리티가 높지는 않지만 우리가 요청한 내용들이 잘 반영되어 있음
- Assistants API 를 사용하면 일반 대화형 모델로 하기 힘든 파일 제작등과 같은 특정 task의 결과물을 얻을 수 있음